# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
# Load training data
training = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data.txt")

# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

#LIBSVM FORMAT

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [7]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [9]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [10]:
# Fit the model
lrModel = lr.fit(training)

In [11]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


Intercept:0.14228558260358093

There is a summary attribute that contains even more info!

In [13]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [15]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
 residuals|
+-------------------+
-11.011130022096554|
 0.9236590911176535|
-4.5957401897776675|
 -20.4201774575836|
-10.339160314788181|
-5.9552091439610555|
-10.726906349283922|
 2.1228071931912336|
 4.077122222293811|
-17.316168071241652|
 -4.593044343959059|
 6.380476690746936|
 11.320566035059846|
-20.721971774534094|
 -2.736692773777401|
 -16.66886934252847|
 8.242186378876315|
-1.3723486332690233|
-0.7060332131264666|
-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154

## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [17]:
all_data = spark.read.format("libsvm").load("/FileStore/tables/sample_linear_regression_data.txt")

all_data.count()

Out[38]: 501

In [18]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [19]:
print("train_data count: {}".format(str(train_data.count())))
print("\n")
print("test_data count: {}".format(str(test_data.count())))

train_data count: 356


test_data count: 145

Now we only train on the train_data

In [21]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [23]:
test_results = correct_model.evaluate(test_data)

In [24]:
test_results.residuals.show(5)
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
 residuals|
+-------------------+
 -20.93644669850324|
 -21.02089730072712|
 -20.33004715469349|
-18.259750635170167|
-18.984962651010857|
+-------------------+
only showing top 5 rows

RMSE: 10.011149865672214

##Deployment

Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [27]:
unlabeled_data = test_data.select('features')

In [28]:
unlabeled_data.show(5)

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows

In [29]:
predictions = correct_model.transform(unlabeled_data)

In [30]:
predictions.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...| -1.901013718416102|
(10,[0,1,2,3,4,5,...|-0.4114904634386843|
(10,[0,1,2,3,4,5,...| 0.5472843650789526|
(10,[0,1,2,3,4,5,...|-0.9085419877935955|
(10,[0,1,2,3,4,5,...|0.13904017811227634|
(10,[0,1,2,3,4,5,...| 1.9007473987127463|
(10,[0,1,2,3,4,5,...| 1.4939227957008505|
(10,[0,1,2,3,4,5,...| 1.7943644989867527|
(10,[0,1,2,3,4,5,...|-1.2432594521795464|
(10,[0,1,2,3,4,5,...| 2.729581534570809|
(10,[0,1,2,3,4,5,...| -2.446913914782189|
(10,[0,1,2,3,4,5,...| -1.713626730886831|
(10,[0,1,2,3,4,5,...| 2.9101982282920082|
(10,[0,1,2,3,4,5,...| -1.01336715338129|
(10,[0,1,2,3,4,5,...|-0.7900711760887542|
(10,[0,1,2,3,4,5,...| 2.8221653732836973|
(10,[0,1,2,3,4,5,...| 1.0587163231968255|
(10,[0,1,2,3,4,5,...|-1.6215442576376118|
(10,[0,1,2,3,4,5,...| 3.302046039601167|
(10,[0,1,2,3,4,5,...|-1.8815836386104186|
+--------------------+-------------------+
only showing top 20 rows

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!